In [1]:
import pandas as pd
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import metrics
from collections import defaultdict
import joblib


import sys

import time
start_time = time.time()


infile='/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/genepromdelta.7/Randomforest/developing/onPBMC/BL14_genepromSM_0.7.txtPBMC_bestref.txt_result_dupindex_binnedstats.pkl_totalformatteddata.pkl'


#gtcol='label2'

##### carefully input the model folder#####

modelfol='/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/genepromdelta.7/Randomforest/developing/onPBMC/BL14_genepromSM_0.7.txt_result_dupindex_binnedstats.pkl_train.pkl_model'


indf=pd.read_pickle(infile)

indf.head()

,index,CD4-others,CD8-others,nB-others,NK-others,Mn-others,mNeu-others,m8-others,DC-others,Eo-others,...,total_cpg,LENhypoCpG_BY_total_cpg,maxscoredCT_beforeCpGweight,maxscore_beforeCpGweight,maxscoredCT,maxscore,deltabasedfragassignment,filename,deltaMedian,deltaMean
0,A00118:116:HJ7VFDSXX:3:1102:19949:17722,-0.061679,-0.066346,-0.026500,-0.095423,-0.053423,0.956731,-0.107808,-0.050551,-0.063654,...,1.0,1.000000,mNeu-others,0.956731,mNeu-others,0.956731,mNeu-others,TWCY-1382-1382-BULK-PBMC_sorted_onlyregionIN_B...,-0.069577,-1.348128e-16
1,A00118:116:HJ7VFDSXX:3:1103:12717:2597,0.739654,0.554423,-0.180038,-0.036808,-0.145577,-0.172859,-0.046141,-0.157782,-0.167295,...,1.0,1.000000,CD4-others,0.739654,CD4-others,0.739654,CD4-others,TWCY-1382-1382-BULK-PBMC_sorted_onlyregionIN_B...,-0.123410,8.128419e-17
2,A00118:116:HJ7VFDSXX:3:1103:4670:35446,0.760705,0.612808,-0.191295,-0.197038,-0.226474,-0.191295,-0.095628,-0.194885,-0.190397,...,1.0,1.000000,CD4-others,0.760705,CD4-others,0.760705,CD4-others,TWCY-1382-1382-BULK-PBMC_sorted_onlyregionIN_B...,-0.191295,-3.766828e-17
3,A00118:116:HJ7VFDSXX:3:1106:14154:33802,-0.189551,-0.081859,-0.212885,-0.170885,-0.048115,1.917269,-0.166038,-0.236577,-0.083654,...,3.0,0.666667,mNeu-others,1.917269,mNeu-others,1.278179,mNeu-others,TWCY-1382-1382-BULK-PBMC_sorted_onlyregionIN_B...,-0.168462,-9.516197e-17
4,A00118:116:HJ7VFDSXX:3:1116:22724:8563,-0.084538,-0.094231,-0.097462,-0.089923,-0.009513,0.963667,-0.081308,-0.034641,-0.035000,...,1.0,1.000000,mNeu-others,0.963667,mNeu-others,0.963667,mNeu-others,TWCY-1382-1382-BULK-PBMC_sorted_onlyregionIN_B...,-0.085346,-7.137148e-17


In [2]:
features=['CD4-others', 'CD8-others', 'nB-others', 'NK-others','Mn-others', 'mNeu-others', 'm8-others', 'DC-others', 'Eo-others','Tr-others', 'm4-others', 'Mg-others', 'Er-others', 'mB-others','LENhypoCpG', 'LENhyperCpG', 'total_cpg', 'LENhypoCpG_BY_total_cpg','deltaMedian', 'deltaMean']

In [3]:
scorecolumns=['CD4-others', 'CD8-others', 'nB-others', 'NK-others','Mn-others', 'mNeu-others', 'm8-others', 'DC-others', 'Eo-others','Tr-others', 'm4-others', 'Mg-others', 'Er-others', 'mB-others']

In [4]:
def calscore(fgrouped,fname):
    celltype_probadded=defaultdict(list)
    
    celltype_proportiondict=defaultdict(list)
    

    for score in scorecolumns:

        tempct=score.replace('-others','')



        indfsingleCT=fgrouped[fgrouped['deltabasedfragassignment']==score].copy()





    
        testX=indfsingleCT[features]

        RDF_classifier =joblib.load(modelfol+"/"+tempct+".pkl")
        
        
        predictedCT=(RDF_classifier.predict(testX)).tolist()
        predictedTARGETCT=predictedCT.count(1)
        predictedNON_TARGETCT=predictedCT.count(0)
        
        totalpredictedCT=len(predictedCT)
        
        if (predictedTARGETCT+predictedNON_TARGETCT) != totalpredictedCT:
            print ("counting error. Exiting")
            sys.exit(1)
        
        
        cellularproportion=1.0*predictedTARGETCT/totalpredictedCT
        
        
        celltype_proportiondict[tempct]=celltype_proportiondict[tempct]+[cellularproportion]
        

        

        predict_test=RDF_classifier.predict_proba(testX)
        
        tempctprob=(predict_test[:,1]).tolist()
        
        tempct_sumprob=sum(tempctprob)
        
        celltype_probadded[tempct]=celltype_probadded[tempct]+[tempct_sumprob]
        
        
 
        
    
    
    celltype_proportiondictDF=pd.DataFrame.from_dict(celltype_proportiondict)
    celltype_proportiondictDF['Mixture']=fname
    
    celltype_probaddeddf=pd.DataFrame.from_dict(celltype_probadded)
    celltype_probaddeddf['Mixture']=fname
    return celltype_proportiondictDF, celltype_probaddeddf
    

In [5]:
filegrouped=indf.groupby('filename')

probaddedallfilelist=[]
ctproportionallfilelist=[]
for name, group in filegrouped:
    tempctproportion,tempprobadded=calscore(group,name)
    
    ctproportionallfilelist.append(tempctproportion)
    probaddedallfilelist.append(tempprobadded)
    

In [6]:
ctproportionallfilelistDF=pd.concat(ctproportionallfilelist)
ctproportionallfilelistDF.set_index('Mixture',inplace=True)
probaddedallfilelistDF=pd.concat(probaddedallfilelist)
probaddedallfilelistDF.set_index('Mixture',inplace=True)

In [7]:
ctproportionallfilelistDFNORMAL=ctproportionallfilelistDF.div(ctproportionallfilelistDF.sum(axis=1), axis=0)

In [8]:
ctproportionallfilelistDF.to_csv(infile+"_RFctprop_CSxOut.txt",sep="\t")
ctproportionallfilelistDFNORMAL.to_csv(infile+"_RFctpropNORMALIZED_CSxOut.txt",sep="\t")

probaddedallfilelistDF.to_csv(infile+"_RFprob_CSxOut.txt",sep="\t")

In [9]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

14.150627851486206
